In [1]:
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from lohrasb.best_estimator import BaseModel
from sklearn.pipeline import Pipeline
from feature_engine.imputation import (
    CategoricalImputer,
    MeanMedianImputer
    )
from category_encoders import OrdinalEncoder
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    f1_score)
from sklearn.metrics import f1_score, make_scorer
from xgboost import *

from lohrasb import logger


2023-07-01 21:10:57,268 :: matplotlib :: matplotlib data path: /Users/hjavedani/Documents/Lohrasb/.venv/lib/python3.10/site-packages/matplotlib/mpl-data
2023-07-01 21:10:57,274 :: matplotlib :: CONFIGDIR=/Users/hjavedani/.matplotlib
2023-07-01 21:10:57,277 :: matplotlib :: interactive is False
2023-07-01 21:10:57,278 :: matplotlib :: platform is darwin


#### Example: Use Adult Data Set (a classification problem)
  
https://archive.ics.uci.edu/ml/datasets/Adult

#### Part 1: Use BestModel in sklearn pipeline


In [2]:
urldata= "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
# column names
col_names=["age", "workclass", "fnlwgt" , "education" ,"education-num",
"marital-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week",
"native-country","label"
]
# read data
data = pd.read_csv(urldata,header=None,names=col_names,sep=',')
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


#### Define labels


In [3]:
data.loc[data['label']=='<=50K','label']=0
data.loc[data['label']==' <=50K','label']=0

data.loc[data['label']=='>50K','label']=1
data.loc[data['label']==' >50K','label']=1

data['label']=data['label'].astype(int)

#### Train test split

In [4]:
X = data.loc[:, data.columns != "label"]
y = data.loc[:, data.columns == "label"]


X_train, X_test, y_train, y_test =train_test_split(X, y, \
     test_size=0.33, stratify=y['label'], random_state=42)


#### Find feature types for later use

In [5]:
int_cols =  X_train.select_dtypes(include=['int']).columns.tolist()
float_cols =  X_train.select_dtypes(include=['float']).columns.tolist()
cat_cols =  X_train.select_dtypes(include=['object']).columns.tolist()


#### Define estimator and set its arguments 


In [6]:
estimator = XGBClassifier()
estimator_params = {
        "booster": ["gbtree","dart"],
        "eval_metric": ["auc"],
        "max_depth": [4, 5],
        "gamma": [0.1, 1.2],
        "subsample": [0.8],

    }
    

In [7]:

kwargs = {  # params for fit method or fit_params 
            'fit_tunegrid_kwargs' :{
            'sample_weight':None,
            },
            # params for RandomSearchCV 
            'tunegrid_search_kwargs' : {
            'estimator':estimator,
            'param_grid':estimator_params,
            'n_jobs':None,
            'cv':KFold(3),
            'early_stopping':None, 
            'scoring':None, 
            'refit':True, 
            'error_score':'raise', 
            'return_train_score':False, 
            'local_dir':'~/ray_results', 
            'name':None, 
            'max_iters':1, 
            'use_gpu':False, 
            'loggers':None, 
            'pipeline_auto_early_stop':True, 
            'stopper':None, 
            'time_budget_s':None, 
            'mode':None,
                }
            }


In [8]:

obj = BaseModel().optimize_by_tunegridsearchcv(kwargs=kwargs)

#### Build sklearn pipeline

In [9]:


pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # category missing values imputers
            ('catimputer', CategoricalImputer(variables=cat_cols)),
            #
            ('catencoder', OrdinalEncoder()),
            # classification model
            ('obj', obj)

 ])


#### Run Pipeline

In [10]:
pipeline.fit(X_train,y_train)
y_pred = pipeline.predict(X_test)


2023-07-01 21:12:00,982 :: dev :: If refit is set to True, the optimal model will be refit on the entire dataset again!
2023-07-01 21:12:00,982 :: dev :: If refit is set to True, the optimal model will be refit on the entire dataset again!


#### Check performance of the pipeline

In [11]:
print('F1 score : ')
print(f1_score(y_test,y_pred))
print('Classification report : ')
print(classification_report(y_test,y_pred))
print('Confusion matrix : ')
print(confusion_matrix(y_test,y_pred))


F1 score : 
0.7138089278264498
Classification report : 
              precision    recall  f1-score   support

           0       0.90      0.94      0.92      8158
           1       0.78      0.66      0.71      2588

    accuracy                           0.87     10746
   macro avg       0.84      0.80      0.82     10746
weighted avg       0.87      0.87      0.87     10746

Confusion matrix : 
[[7663  495]
 [ 877 1711]]


#### Part 2:  Use BestModel as a standalone estimator 

In [12]:
X_train, X_test, y_train, y_test =train_test_split(X, y, \
     test_size=0.33, stratify=y['label'], random_state=42)

#### Transform features to make them ready for model input

In [13]:
transform_pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # category missing values imputers
            ('catimputer', CategoricalImputer(variables=cat_cols)),
            #
            ('catencoder', OrdinalEncoder()),
            # classification model

 ])

#### Transform X_train and X_test

In [14]:
X_train=transform_pipeline.fit_transform(X_train,y_train)
X_test=transform_pipeline.transform(X_test)


#### Train model and predict

In [15]:
obj.fit(X_train,y_train)
y_pred = obj.predict(X_test)

2023-07-01 21:13:01,565 :: dev :: If refit is set to True, the optimal model will be refit on the entire dataset again!
2023-07-01 21:13:01,565 :: dev :: If refit is set to True, the optimal model will be refit on the entire dataset again!


#### Check performance of the pipeline

In [16]:
print('F1 score : ')
print(f1_score(y_test,y_pred))
print('Classification report : ')
print(classification_report(y_test,y_pred))
print('Confusion matrix : ')
print(confusion_matrix(y_test,y_pred))

F1 score : 
0.7138089278264498
Classification report : 
              precision    recall  f1-score   support

           0       0.90      0.94      0.92      8158
           1       0.78      0.66      0.71      2588

    accuracy                           0.87     10746
   macro avg       0.84      0.80      0.82     10746
weighted avg       0.87      0.87      0.87     10746

Confusion matrix : 
[[7663  495]
 [ 877 1711]]


In [17]:
obj.get_best_estimator()

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='auc', feature_types=None, gamma=1.2, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=4, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [18]:
obj.best_estimator

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='auc', feature_types=None, gamma=1.2, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=4, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0, ...)

#### Get fitted grid search object and its attributes

In [19]:
GridSearchObj = obj.get_optimized_object()
GridSearchObj.cv_results_

{'params': [{'booster': 'gbtree',
   'eval_metric': 'auc',
   'max_depth': 4,
   'gamma': 0.1,
   'subsample': 0.8},
  {'booster': 'dart',
   'eval_metric': 'auc',
   'max_depth': 4,
   'gamma': 0.1,
   'subsample': 0.8},
  {'booster': 'gbtree',
   'eval_metric': 'auc',
   'max_depth': 4,
   'gamma': 1.2,
   'subsample': 0.8},
  {'booster': 'dart',
   'eval_metric': 'auc',
   'max_depth': 4,
   'gamma': 1.2,
   'subsample': 0.8},
  {'booster': 'gbtree',
   'eval_metric': 'auc',
   'max_depth': 5,
   'gamma': 0.1,
   'subsample': 0.8},
  {'booster': 'dart',
   'eval_metric': 'auc',
   'max_depth': 5,
   'gamma': 0.1,
   'subsample': 0.8},
  {'booster': 'gbtree',
   'eval_metric': 'auc',
   'max_depth': 5,
   'gamma': 1.2,
   'subsample': 0.8},
  {'booster': 'dart',
   'eval_metric': 'auc',
   'max_depth': 5,
   'gamma': 1.2,
   'subsample': 0.8}],
 'split0_test_score': array([0.86207371, 0.86386139, 0.86193619, 0.86234873, 0.86331133,
        0.86111111, 0.86248625, 0.86138614]),
 'spli